# Statistik-Projekt HS25 – Notebook 07: Hypothesentest I (Tageszeit)
## Forschungsfrage
Unterscheidet sich die Ankunftsverspätung von Zügen signifikant je nach Tageszeit?
Wir vermuten: **Rush-Hour (Morgen/Abend) > Nebenverkehrszeiten**.

## Hypothesen
* $H_0$: Die Verteilung der Verspätungen ist in allen Zeitbändern gleich. ($\mu_{Nacht} = \mu_{Morgen} = \dots$)
* $H_1$: Es gibt mindestens zwei Zeitbänder mit unterschiedlichen Verteilungen.

## Methode
Da wir mehr als zwei Gruppen haben (5 Zeitbänder) und die Daten nicht normalverteilt sind (siehe N03/N05), nutzen wir den **Kruskal-Wallis-Test**.
Falls dieser signifikant ist, führen wir **Post-hoc-Tests** (paarweise Vergleiche) mit Korrektur durch.

## Input
* Datei: `../data/processed/istdata_clean_extended.parquet`

In [1]:
import polars as pl
import pandas as pd
import numpy as np
from scipy.stats import kruskal
from statsmodels.stats.multitest import multipletests
from itertools import combinations
from pathlib import Path

# 1. Konfiguration
INPUT_PATH = "../../data/processed/istdata_clean_extended.parquet"

# 2. Daten laden
# Wir laden die vorbereitete Datei. Die Spalte 'time_band' wurde bereits in N01 erstellt!
print(f"Lade Daten aus {INPUT_PATH}...")
df = pl.read_parquet(INPUT_PATH)

# Sample ziehen für Performance (Kruskal-Wallis mit 60 Mio. Daten ist langsam und p-Werte werden unlesbar klein)
# Ein Sample von 200.000 ist statistisch absolut repräsentativ.
df_sample = df.sample(n=200000, seed=42).to_pandas()
print(f"Arbeite mit Stichprobe: {len(df_sample)} Zeilen.")

# Check der Gruppen
print("\nVerteilung der Zeitbänder im Sample:")
print(df_sample["time_band"].value_counts())

Lade Daten aus ../../data/processed/istdata_clean_extended.parquet...
Arbeite mit Stichprobe: 200000 Zeilen.

Verteilung der Zeitbänder im Sample:
time_band
Tagesverkehr    64105
Abendpeak       44998
Morgenpeak      44104
Spätabend       33779
Nacht           13014
Name: count, dtype: int64


In [2]:
# Daten in Listen formatieren für Scipy
# Wir nutzen die Spalte 'time_band' als Gruppierung
bands = ["Nacht", "Morgenpeak", "Tagesverkehr", "Abendpeak", "Spätabend"]
groups = [df_sample[df_sample["time_band"] == b]["arr_delay_min"].values for b in bands]

# Test durchführen
stat, p_value = kruskal(*groups)

print(f"--- Kruskal-Wallis Test ---")
print(f"H-Statistik: {stat:.2f}")
print(f"p-Wert:      {p_value:.4e}") # Wissenschaftliche Notation

if p_value < 0.05:
    print("\nERGEBNIS: H0 abgelehnt. Es gibt signifikante Unterschiede zwischen den Tageszeiten.")
else:
    print("\nERGEBNIS: H0 angenommen. Keine signifikanten Unterschiede.")

# Effektstärke (Epsilon-Squared)
# Formel: (H - k + 1) / (n - k)
n = len(df_sample)
k = len(bands)
epsilon2 = (stat - k + 1) / (n - k)

print(f"Effektstärke (Epsilon²): {epsilon2:.4f}")
print("(Interpretation: <0.01 = vernachlässigbar, 0.01-0.06 = klein, 0.06-0.14 = mittel, >0.14 = gross)")

--- Kruskal-Wallis Test ---
H-Statistik: 1277.73
p-Wert:      2.2362e-275

ERGEBNIS: H0 abgelehnt. Es gibt signifikante Unterschiede zwischen den Tageszeiten.
Effektstärke (Epsilon²): 0.0064
(Interpretation: <0.01 = vernachlässigbar, 0.01-0.06 = klein, 0.06-0.14 = mittel, >0.14 = gross)


In [3]:
print("--- Post-hoc Tests (Paarweise Vergleiche) ---")

# Alle möglichen Paare bilden
pairs = list(combinations(bands, 2))
p_values = []
pair_names = []

# Mann-Whitney U Test für jedes Paar
from scipy.stats import mannwhitneyu

for b1, b2 in pairs:
    group1 = df_sample[df_sample["time_band"] == b1]["arr_delay_min"]
    group2 = df_sample[df_sample["time_band"] == b2]["arr_delay_min"]
    
    # Test
    _, p = mannwhitneyu(group1, group2, alternative='two-sided')
    
    p_values.append(p)
    pair_names.append(f"{b1} vs {b2}")

# P-Wert Korrektur (Bonferroni oder Benjamini-Hochberg) gegen alpha-Fehler-Kumulierung
reject, p_corrected, _, _ = multipletests(p_values, method='fdr_bh')

# Ergebnisse als Tabelle
results_df = pd.DataFrame({
    "Vergleich": pair_names,
    "p-Wert (raw)": p_values,
    "p-Wert (korrigiert)": p_corrected,
    "Signifikant?": reject
})

# Wir zeigen nur die signifikanten Unterschiede an und sortieren nach Relevanz
display(results_df.sort_values("p-Wert (korrigiert)"))

--- Post-hoc Tests (Paarweise Vergleiche) ---


,Vergleich,p-Wert (raw),p-Wert (korrigiert),Signifikant?
7,Tagesverkehr vs Abendpeak,1.826498e-199,1.826498e-198,True
4,Morgenpeak vs Tagesverkehr,1.964784e-166,9.823919e-166,True
9,Abendpeak vs Spätabend,4.658524e-71,1.552841e-70,True
6,Morgenpeak vs Spätabend,1.811662e-54,4.529155e-54,True
2,Nacht vs Abendpeak,1.980495e-49,3.960990e-49,True
0,Nacht vs Morgenpeak,5.784166e-40,9.640277e-40,True
8,Tagesverkehr vs Spätabend,1.161725e-15,1.659608e-15,True
1,Nacht vs Tagesverkehr,2.003909e-04,2.504886e-04,True
5,Morgenpeak vs Abendpeak,4.562673e-03,5.069636e-03,True
3,Nacht vs Spätabend,8.941997e-02,8.941997e-02,False


In [4]:
# Kurz die Mediane berechnen, um die Richtung des Unterschieds zu sehen
medians = df_sample.groupby("time_band", observed=False)["arr_delay_min"].median().sort_values(ascending=False)

print("--- Ranking der Unpünktlichkeit (Median) ---")
print(medians)

--- Ranking der Unpünktlichkeit (Median) ---
time_band
Abendpeak       0.716667
Morgenpeak      0.700000
Spätabend       0.566667
Nacht           0.550000
Tagesverkehr    0.516667
Name: arr_delay_min, dtype: float64
